In [1]:
import numpy as np
import os
import re
import io
import json
import pypdf
import langchain
import openai

In [2]:
api_key = json.load(open('./data/creds/gpt_id.json'))['api_key']
openai.api_key = api_key

In [3]:
def extract_text_from_pdf(pdf_path: str) -> str:
    '''
    Función para extraer texto de un pdf y limpiarlo.
    Devuelve una lista de str, cada una es una página del pdf.
    '''
    # Abrimos el pdf
    with open(pdf_path, 'rb') as f:
        pdf = pypdf.PdfReader(f)
        # Obtenemos el número de páginas
        num_pags = len(pdf.pages)
        count = 0
        text = []
        # Iteramos sobre las páginas
        for pag in pdf.pages:
        #while count < num_pags:
            #pag = pdf.pages[count]
            count +=1
            texto_pagina = pag.extract_text()
            # texto_pagina = limpieza_texto(texto_pagina)
            text.append(texto_pagina)
    return text

In [4]:
resultado = extract_text_from_pdf('./data/16089013.pdf')

In [28]:
resultado[0]

"DATOSGENERALESDEIDENTIFICACIÓN EINFORMACIÓN\nCOMPLEMENTARIA REQUERIDAENLALEGISLACIÓNESPAÑOLA\n(Aplicaciónderesultadosyperíodomediodepagoaproveedores)IDA1\nIDENTIFICACIÓNDELAEMPRESA\nNIF:\nLEI:01010B85328177\n01009Formajurídica:SA:\nOtras:01011SL:01012\n01013\nSoloparalasempresasquedispongandecódigoLEI(LegalEntityIdentifier)\nDenominaciónsocial:\nDomiciliosocial:01020BUBOKPUBLISHING,S.L.\n01022CLBELEN13\nMunicipio:01023MADRID Provincia:01025MADRID\nCódigopostal:0102428004 Teléfono:01031\nDireccióndee-maildecontactodelaempresa\nPertenenciaaungrupodesociedades:\nSociedaddominantedirecta:\nSociedaddominanteúltimadelgrupo:\nACTIVIDAD\nActividadprincipal:\nCódigoCNAE:02009\n0200101037\nDENOMINACIÓNSOCIAL NIF\n01041 01040\n01061 01060\nEDICIONDELIBROS (1)\n5811 (1)\nPERSONALASALARIADO\na)Númeromediodepersonasempleadasenelcursodelejercicio,portipodecontratoyempleocondiscapacidad:\nEJERCICIO2021(2)\nFIJO(4):\nNOFIJO(5):\nDelcual:Personasempleadascondiscapacidadmayoroigualal33%(ocalificaciónequ

In [ ]:
with open('./data/rovi-sostenibilidad-2020.pdf', 'rb') as f:
    pdf = pypdf.PdfReader(f)
    # Obtenemos el número de páginas
    num_pags = len(pdf.pages)
    count = 0
    text = []
    # Iteramos sobre las páginas
    for pag in pdf.pages:
    #while count < num_pags:
        #pag = pdf.pages[count]
        count +=1
        texto_pagina = pag.extract_text()
        # texto_pagina = limpieza_texto(texto_pagina)
        text.append(texto_pagina)

In [5]:
with open('./data/rovi-sostenibilidad-2020.pdf', 'rb') as f:
    pdf = pypdf.PdfReader(f)

In [6]:
import tabula

In [70]:
tabla_ej = tabula.read_pdf('./data/rovi-sostenibilidad-2020.pdf', pages=14)[0]
tabla_ej

,KEY FIGURES,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,102-7
0,(million euros),2020.000,2019.00,2018.000,2017.000,NaN
1,Total revenue,421.100,382.50,304.800,277.400,NaN
2,EBITDA,94.200,60.90,29.500,29.900,NaN
3,Net financial debt,19.800,15.90,-62.800,1.100,NaN
4,Employees,1.419,1.31,1.224,1.191,NaN


In [127]:
def tabla_a_texto(tabla):
    '''
    Función para convertir una tabla de pandas en un texto.
    La idea es identificar los nombres de columna e índices correctos y
    a partir de ahí generar un texto que pueda ser procesado por el modelo.
    '''
    tabla = tabla.copy()
    # Tamaño mínimo de tabla para que sea válida = 2x2
    if sum(tabla.shape) < 4:
        return ''
    # Asumimos que el primer elemento es el título
    titulo = tabla.columns[0] 
    
    # Asumimos que la primera columna es el índice y la eliminamos:
    tabla.index = tabla[tabla.columns[0]].values
    tabla.drop(columns=tabla.columns[0], inplace=True)
    
    # Eliminamos las filas y columnas que no tienen datos:
    tabla.replace('', np.nan, inplace=True)
    tabla.dropna(axis=0, how='all', inplace=True)
    tabla.dropna(axis=1, how='all', inplace=True)
    
    # Check si las columnas son años:
    col_años = False
    years_txt=[str(i) for i in range(2015,2022)]
    years_int=[i for i in range(2015,2022)]
    years = set(years_txt+years_int)
    cruce = set(tabla.columns).intersection(set(years))
    if len(cruce) > 0: col_años=True
    
    # Si no son años las columnas, buscamos filas que sean años:
    if not col_años:
        for i in tabla.iterrows():
            #print(i[1].values)
            try:
                cruce = set(i[1].values).intersection(set(years))
            except:
                cruce=[]
            if len(cruce)>0: # Si encontramos una fila con años:
                # Asignamos los años a las columnas:
                tabla.columns = i[1].values
                contexto = None
                try: contexto = i[1].name
                # Drop de la fila:
                tabla.drop(i[0], inplace=True)
                break
    # Los procesos anteriores pueden haber dejado filas vacías, las eliminamos:
    tabla.replace('', np.nan, inplace=True)
    tabla.dropna(axis=0, how='all', inplace=True)
    tabla.dropna(axis=1, how='all', inplace=True)
    # Pasamos a texto:
    texto = ''
    for i in tabla.items():
        add =f'1-Columna: {i[0]} 2-Datos: [{i[1]}]'
        if contexto:
            add = f'A-Contexto: {contexto} B-Contenido:{add}'
        add = add.replace('  ',' ').replace('\n','; ').replace('  ','')
        texto += f';  {titulo}: {add}'
    return texto

In [85]:
import itertools

In [88]:
list(itertools.chain(*[[i, str(i)] for i in range(2015,2022)]))

[2015,
 '2015',
 2016,
 '2016',
 2017,
 '2017',
 2018,
 '2018',
 2019,
 '2019',
 2020,
 '2020',
 2021,
 '2021']

In [128]:
test = tabla_a_texto(tabla_ej)

In [129]:
test

';  KEY FIGURES: A-Contexto: (million euros) B-Contenido:1-Columna: 2020.0 2-Datos: [Total revenue 421.100; EBITDA 94.200; Net financial debt 19.800; Employees1.419; Name: 2020.0, dtype: float64];  KEY FIGURES: A-Contexto: (million euros) B-Contenido:1-Columna: 2019.0 2-Datos: [Total revenue 382.50; EBITDA 60.90; Net financial debt 15.90; Employees1.31; Name: 2019.0, dtype: float64];  KEY FIGURES: A-Contexto: (million euros) B-Contenido:1-Columna: 2018.0 2-Datos: [Total revenue 304.800; EBITDA 29.500; Net financial debt-62.800; Employees1.224; Name: 2018.0, dtype: float64];  KEY FIGURES: A-Contexto: (million euros) B-Contenido:1-Columna: 2017.0 2-Datos: [Total revenue 277.400; EBITDA 29.900; Net financial debt 1.100; Employees1.191; Name: 2017.0, dtype: float64]'

In [101]:
tabla_a_texto(tabla_ej)

                     2020.0  2019.0   2018.0   2017.0
KEY FIGURES                                          
Total revenue       421.100  382.50  304.800  277.400
EBITDA               94.200   60.90   29.500   29.900
Net financial debt   19.800   15.90  -62.800    1.100
Employees             1.419    1.31    1.224    1.191
                     2020.0  2019.0   2018.0   2017.0
KEY FIGURES                                          
Total revenue       421.100  382.50  304.800  277.400
EBITDA               94.200   60.90   29.500   29.900
Net financial debt   19.800   15.90  -62.800    1.100
Employees             1.419    1.31    1.224    1.191


'KEY FIGURES;  KEY FIGURES: 1-Columna: 2020.0 2-Datos: [KEY FIGURES; Total revenue 421.100; EBITDA 94.200; Net financial debt 19.800; Employees1.419; Name: 2020.0, dtype: float64];  KEY FIGURES: 1-Columna: 2019.0 2-Datos: [KEY FIGURES; Total revenue 382.50; EBITDA 60.90; Net financial debt 15.90; Employees1.31; Name: 2019.0, dtype: float64];  KEY FIGURES: 1-Columna: 2018.0 2-Datos: [KEY FIGURES; Total revenue 304.800; EBITDA 29.500; Net financial debt-62.800; Employees1.224; Name: 2018.0, dtype: float64];  KEY FIGURES: 1-Columna: 2017.0 2-Datos: [KEY FIGURES; Total revenue 277.400; EBITDA 29.900; Net financial debt 1.100; Employees1.191; Name: 2017.0, dtype: float64]'

In [14]:
tabla_ej.index = tabla_ej[tabla_ej.columns[0]]
tabla_ej.drop(columns=tabla_ej.columns[0], inplace=True)

In [49]:
f'1-Columna: {i[0]} 2-Datos: [{i[1]}]'.replace('  ',' ').replace('\n','; ').replace('  ','')

'1-Columna: Unnamed: 0 2-Datos: [KEY FIGURES; (million euros)2020.000; Total revenue 421.100; EBITDA 94.200; Net financial debt 19.800; Employees1.419; Name: Unnamed: 0, dtype: float64]'

In [34]:
for i in tabla_ej.iteritems():
    print(i)
    break

('Unnamed: 0', KEY FIGURES
(million euros)       2020.000
Total revenue          421.100
EBITDA                  94.200
Net financial debt      19.800
Employees                1.419
Name: Unnamed: 0, dtype: float64)


C:\Users\Faji\AppData\Local\Temp\ipykernel_2796\1429811112.py:1: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i in tabla_ej.iteritems():


In [38]:
str(i)

"('Unnamed: 0', KEY FIGURES\n(million euros)       2020.000\nTotal revenue          421.100\nEBITDA                  94.200\nNet financial debt      19.800\nEmployees                1.419\nName: Unnamed: 0, dtype: float64)"

In [20]:
tabla_ej

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,102-7
KEY FIGURES,,,,,
(million euros),2020.000,2019.00,2018.000,2017.000,NaN
Total revenue,421.100,382.50,304.800,277.400,NaN
EBITDA,94.200,60.90,29.500,29.900,NaN
Net financial debt,19.800,15.90,-62.800,1.100,NaN
Employees,1.419,1.31,1.224,1.191,NaN


In [21]:
def limpieza_passages(texto):
  '''
  Dado un texto realiza una limpieza caracteres especiales, transformar a minusculas y eleiminar urls
  Args:
        texto (str): texto a limpiar

    Returns:
        text(str): texto limpio
  '''
  texto=texto.replace("\n"," ")
  texto=texto.replace("\t"," ")
  texto=texto.replace("\\r"," ")
  texto=texto.replace("\r"," ")
  texto=texto.replace(" - ","")
  texto=texto.replace("- ","")
  texto=texto.replace("-","")
  texto=texto.replace(" -","")
  texto=texto.replace(">"," ")
  texto=texto.lower()
  texto = re.sub(r'^https?:\/\/.*[\r\n]*', '', texto, flags=re.MULTILINE)
 
  return texto

def columnas_son_años(columnas):
  '''
  Dado una lista de columnas determina si son años 
  Args:
        columnas list(str): nombre de las columnas de una tabla
       
  Return:
      Bool: indica si son años o no
  '''
  años=['2019','2020','2021','2018','2017']
  años = set(años)
  columnas = set(columnas)
  son=False
  interseccion= set(columnas).intersection(años)
  interseccion = list(interseccion)

  
  if len(interseccion)>1:
    son=True
  return son 

def tabla_to_lines(df,año_actual):
  '''
  Extrae la informacion de una tabla a un texto. Ademas si las columnas son años, solo extrae la informacion del año que estamos estudiando
  Args:
        df Dataframe: tabla en formato dataframe de la que queremos extraer la informacion.
        año_actual str: año del que hacemos el estudio
       
  Return:
      txt: con la informacion de la tabla
  '''
  columnas=list(df.columns)
  num_columnas=df.shape[1]
  num_filas=df.shape[0]
  if num_filas>0 and num_columnas>0:
    titulo=titulo=columnas[0]
    filas=list(df.iloc[0:,0])
    texto=''
    if columnas_son_años(columnas[1:])==False:
      for i in range(0,num_filas):
        
        for j in range(1,num_columnas):
          
          dato=df.iloc[i,j]
          if str(dato)!='Nan' and str(dato)!='na' and str(dato)!='NaN' and str(dato)!='nan':
            if str(columnas[j])[0:8]!='Unnamed:':
              if str(titulo) !='Unnamed: 0':
                if str(filas[i])!='nan':
                  texto=texto+' '+str(titulo)+' '+str(dato)+' '+str(filas[i])+' '+str(columnas[j])+'             '
                else: 
                  texto=texto+' '+str(titulo)+' '+str(dato)+' '+str(columnas[j])+'             '
              else:
                if str(filas[i])!='nan':
                  texto=texto+' '+str(dato)+' '+str(filas[i])+' '+str(columnas[j])+'             '
                else: 
                  texto=texto+' '+str(dato)+' '+str(columnas[j])+'             '
              
            else:
              if str(titulo) !='Unnamed: 0':
                if str(filas[i])!='nan':
                  texto=texto+' '+str(titulo)+' '+str(dato)+' '+str(filas[i])+'             '
                else: 
                  texto=texto+' '+str(titulo)+' '+str(dato)+'             '
              else:
                if str(filas[i])!='nan':
                  texto=texto+' '+str(dato)+' '+str(filas[i])+'             '
                else: 
                  texto=texto+' '+str(dato)+'                  '
    else:
      indice_actual=columnas.index(año_actual)
      for i in range(0,num_filas):
        
        
          
          dato=df.iloc[i,indice_actual]
          if str(dato)!='Nan' and str(dato)!='na' and str(dato)!='NaN' and str(dato)!='nan':
            
              if str(titulo) !='Unnamed: 0':
                if str(filas[i])!='nan':

                  texto=texto+' '+str(titulo)+' '+str(dato)+' '+str(filas[i])+' '+str(columnas[indice_actual])+'             '
                else: 
                  texto=texto+' '+str(titulo)+' '+str(dato)+' '+str(columnas[indice_actual])+'             '
                
              else:
                if str(filas[i])!='nan':
                  texto=texto+' '+str(dato)+' '+str(filas[i])+' '+str(columnas[indice_actual])+'             '
                else: 
                  texto=texto+' '+str(dato)+' '+str(columnas[indice_actual])+'                  '
             
    return limpieza_passages(texto)

In [22]:
tabla_to_lines(tabula.read_pdf('./data/rovi-sostenibilidad-2020.pdf', pages=14)[0],'2020')

' key figures 2020.0 (million euros)              key figures 2019.0 (million euros)              key figures 2018.0 (million euros)              key figures 2017.0 (million euros)              key figures 421.1 total revenue              key figures 382.5 total revenue              key figures 304.8 total revenue              key figures 277.4 total revenue              key figures 94.2 ebitda              key figures 60.9 ebitda              key figures 29.5 ebitda              key figures 29.9 ebitda              key figures 19.8 net financial debt              key figures 15.9 net financial debt              key figures 62.8 net financial debt              key figures 1.1 net financial debt              key figures 1.419 employees              key figures 1.31 employees              key figures 1.224 employees              key figures 1.191 employees             '

In [26]:
columnas_son_años(tabla_ej.columns)

False

In [37]:
tabla_ej.iloc[0,:].values

array(['(million euros)', 2020.0, 2019.0, 2018.0, 2017.0, nan],
      dtype=object)

In [39]:
set(tabla_ej.iloc[0,:].values).intersection(set([2019,2020,2021,2018,2017]))

{2017, 2018, 2019, 2020}

In [41]:
int(años)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [ ]:
def filas_son_años2(df):
  '''
  xx
  '''
  years_txt=['2019','2020','2021','2018','2017']
  years_int=[2019,2020,2021,2018,2017]
  years = set(years_txt+years_int)
  columnas = set(columnas)
  son=False
  interseccion= set(columnas).intersection(years)
  interseccion = list(interseccion)
  
  if len(interseccion)>1:
    son=True
    return son
  else:

In [58]:
years_txt=['2019','2020','2021','2018','2017']
years_int=[2019,2020,2021,2018,2017]
years = set(years_txt+years_int)

In [61]:
for i in tabla_ej.iterrows():
    #print(i[1].values)
    try:
        cruce = set(i[1].values).intersection(set(years))
        if len(cruce)>0:
            return i[0]
    break

{2017.0, 2018.0, 2019.0, 2020.0}


In [71]:
tabla_ej.iloc[:,0]

0       (million euros)
1         Total revenue
2                EBITDA
3    Net financial debt
4             Employees
Name: KEY FIGURES, dtype: object

In [72]:
tabla_ej.set_index(tabla_ej.iloc[:,0].values)

,KEY FIGURES,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,102-7
(million euros),(million euros),2020.000,2019.00,2018.000,2017.000,NaN
Total revenue,Total revenue,421.100,382.50,304.800,277.400,NaN
EBITDA,EBITDA,94.200,60.90,29.500,29.900,NaN
Net financial debt,Net financial debt,19.800,15.90,-62.800,1.100,NaN
Employees,Employees,1.419,1.31,1.224,1.191,NaN
